# Autograd：自动求导
PyTorch 中所有神经网络的核心是autograd包。我们首先简单介绍一下这个包,然后训练我们的第一个神经网络.

autograd包为张量上的所有操作提供了自动求导。它是一个运行时定义的框架,这意味着反向传播是根据你的代码如何运行来定义,并且每次迭代可以不同.

## Tensor
- `torch.Tensor`是`autograd`包的核心类。如果你设置它的属性`.requires_grad`为true，它开始跟踪所有的操作。一旦完成前向计算,可以通过`.backward()`方法来自动计算所有的梯度，这个张量的梯度被计算放入`.grad`属性中
- 为了阻止一个张量跟踪历史，可以调用`.detach()`将它从计算历史中分离出来，并防止未来的计算被跟踪。
- 为了防止跟踪历史（和使用内存），你还可以将代码块包装在`with torch.no_grad():`中。这在评估一个模型时特别有用，因为模型可能具有与`requires_grad=True`的可训练参数，但是我们不需要梯度。
- 还有一个对`autograd`实现非常重要的类——函数。tensor和function相互连接并建立一个非循环图，它编码了一个完整的计算历史，每一个变量都有一个`.grad_fn`属性，它引用了一个已经创建了tensor的函数（除了由用户创建的函数——grad_fn为None）

如果想要计算导数，可以对tensor调用`.backward()`。如果tensor是标量，比如是一个值，就不需要为`backward()`指定任何参数。但如果有多个元素，就需要指定一个`gradient`参数，这是一个匹配形状的张量。

In [1]:
#创建一个张量，并设置跟踪所有操作
import torch
x = torch.ones(2, 2, requires_grad=True)
print(x)

TypeError: torch.ones received an invalid combination of arguments - got (int, int, requires_grad=bool), but expected one of:
 * (int ... size)
      didn't match because some of the keywords were incorrect: requires_grad
 * (torch.Size size)


In [ ]:
#进行一个操作
y = x + 2
print(y)

In [ ]:
# y是由一个操作产生的,所以它有一个grad_fn属性
print(y.grad_fn)

In [ ]:
# 对y进行更多的操作
z = y * y * 3
out = z.mean()

print(z, out)

.requires_grad_(…)改变一个现有张量的requires_grad标志，如果没有给出，输入标志默认为True。

In [ ]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

## Gradients
现在做反向传播，因为`out`包含一个标量，`out.backward()`与`out.backward(torch.tensor(1))`相似。

In [ ]:
out.backward()
print(x.grad)

$y = x + 2$，$z = y^2 \times 2$，$out = \frac 1 4 z$；则$\frac {\partial out} {\partial x} = \frac 1 4 \cdot 6y \cdot 1 = 1.5y$

可以使用aotograd做很多疯狂的事！

In [ ]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000: #二范数
    y = y * 2

print(y)

In [ ]:
gradients = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(gradients) 

print(x.grad)

前面其实是$f = 2^n y$，求导$\frac {\partial f} {\partial x} = 2^n * 2$

还可以通过使用`torch.no_grad():`包装代码块来阻止autograd跟踪`require_grad = True`的tensor上的历史记录

In [ ]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

## 稍后阅读:
autograd和function的文档在[http://pytorch.org/docs/autograd](http://pytorch.org/docs/autograd)